In [1]:
 from pyspark.sql import SparkSession
 import getpass
 username = getpass.getuser()
 spark= SparkSession. \
 builder. \
 config('spark.ui.port','0'). \
 config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
 enableHiveSupport(). \
 master('yarn'). \
 getOrCreate()

In [2]:
spark

Question 1:
Use the dataset given in HDFS (path : /public/trendytech/retail_db/products) and Write the spark program using Dataframes for the below tasks.

To navigate and get glimpse of the the data in hdfs

hadoop fs -ls <path>
    
hadoop fs -head<Path>     

In [3]:
structure = ('ProductID string, Category string, ProductName string, Description string, Price float,ImageURL string')

In [4]:
# Reading the file from hdfs
main_df = spark.read \
.format('csv')\
.schema(structure)\
.load("/public/trendytech/retail_db/products/part-00000")

In [5]:
main_df.limit(5).show(truncate = False)

+---------+--------+---------------------------------------------+-----------+------+-------------------------------------------------------------------------------------+
|ProductID|Category|ProductName                                  |Description|Price |ImageURL                                                                             |
+---------+--------+---------------------------------------------+-----------+------+-------------------------------------------------------------------------------------+
|1        |2       |Quest Q64 10 FT. x 10 FT. Slant Leg Instant U|null       |59.98 |http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy|
|2        |2       |Under Armour Men's Highlight MC Football Clea|null       |129.99|http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat     |
|3        |2       |Under Armour Men's Renegade D Mid Football Cl|null       |89.99 |http://images.acmesports.sports/Under+Armour+Men%27s+Re

In [6]:
# 1.1 Find the total number of products in the given dataset.
main_df.count()

1345

In [7]:
# 1.2 Find the number of unique categories of products in the given dataset.
main_df.select('Category').distinct().count()

55

In [8]:
# 1.3 Find the top 5 most expensive products based on their price, along with their product name, category, and image URL.
main_df.select('ProductName','Category','price','imageURL').orderBy('price',ascending=False).limit(5).show(truncate = False)

+------------------------------------------------+--------+-------+-----------------------------------------------------------------------------------+
|ProductName                                     |Category|price  |imageURL                                                                           |
+------------------------------------------------+--------+-------+-----------------------------------------------------------------------------------+
|SOLE E35 Elliptical                             |10      |1999.99|http://images.acmesports.sports/SOLE+E35+Elliptical                                |
|SOLE F85 Treadmill                              |4       |1799.99|http://images.acmesports.sports/SOLE+F85+Treadmill                                 |
|SOLE F85 Treadmill                              |10      |1799.99|http://images.acmesports.sports/SOLE+F85+Treadmill                                 |
|SOLE F85 Treadmill                              |22      |1799.99|http://images.acmespo

In [9]:
# 1.4. Find the number of products in each category that have a price greater than $100. Display the results in a tabular format that shows the category
# name and the number of products that satisfy the condition

main_df.filter('price> 100').groupBy('Category').count().withColumnRenamed('count','no_prd').orderBy('no_prd',descending=True).limit(10).show()

+--------+------+
|Category|no_prd|
+--------+------+
|      13|     1|
|      21|     1|
|      56|     2|
|      24|     3|
|      12|     3|
|      22|     4|
|      42|     4|
|      25|     4|
|      58|     4|
|      33|     4|
+--------+------+



In [10]:
#  1.5. What are the product names and prices of products that have a price greater than $200 and belong to category 5?

In [11]:
main_df.select('Productname','price').filter('price > 200 and Category = 5').show(truncate=False)

+------------------------------------------------+------+
|Productname                                     |price |
+------------------------------------------------+------+
|"Goaliath 54"" In-Ground Basketball Hoop with P"|499.99|
|Fitness Gear 300 lb Olympic Weight Set          |209.99|
|Teeter Hang Ups NXT-S Inversion Table           |299.99|
+------------------------------------------------+------+



Question 2  Write the spark program using Dataframes for the below tasks

In [13]:
# cust_id,cust_fname,cust_lname,cust_email,cust_password,cust_street,cust_city,cust_state,cust_zipcode

In [14]:
schema = ('cust_id string,cust_fname string,cust_lname string ,cust_email string,cust_password string,cust_street string,cust_city string,cust_state string,cust_zipcode string')

In [15]:
customers_main_df = spark.read \
.format('csv') \
.schema(schema) \
.load('/public/trendytech/retail_db/customers/part-00000')

In [16]:
customers_main_df.show()

+-------+-----------+----------+----------+-------------+--------------------+-------------+----------+------------+
|cust_id| cust_fname|cust_lname|cust_email|cust_password|         cust_street|    cust_city|cust_state|cust_zipcode|
+-------+-----------+----------+----------+-------------+--------------------+-------------+----------+------------+
|      1|    Richard| Hernandez| XXXXXXXXX|    XXXXXXXXX|  6303 Heather Plaza|  Brownsville|        TX|       78521|
|      2|       Mary|   Barrett| XXXXXXXXX|    XXXXXXXXX|9526 Noble Embers...|    Littleton|        CO|       80126|
|      3|        Ann|     Smith| XXXXXXXXX|    XXXXXXXXX|3422 Blue Pioneer...|       Caguas|        PR|       00725|
|      4|       Mary|     Jones| XXXXXXXXX|    XXXXXXXXX|  8324 Little Common|   San Marcos|        CA|       92069|
|      5|     Robert|    Hudson| XXXXXXXXX|    XXXXXXXXX|10 Crystal River ...|       Caguas|        PR|       00725|
|      6|       Mary|     Smith| XXXXXXXXX|    XXXXXXXXX|3151 Sl

In [17]:
#  3.1. Find the total number of customers in each state.
customers_main_df.groupBy('cust_state').count().withColumnRenamed('count','no_cust').orderBy('no_cust',ascending = False).show()

+----------+-------+
|cust_state|no_cust|
+----------+-------+
|        PR|   4771|
|        CA|   2012|
|        NY|    775|
|        TX|    635|
|        IL|    523|
|        FL|    374|
|        OH|    276|
|        PA|    261|
|        MI|    254|
|        NJ|    219|
|        AZ|    213|
|        GA|    169|
|        MD|    164|
|        NC|    150|
|        VA|    136|
|        CO|    122|
|        OR|    119|
|        MA|    113|
|        TN|    104|
|        NV|    103|
+----------+-------+
only showing top 20 rows



In [18]:
#  3.2. Find the top 5 most common last names among the customers.
# Approach - group based on last names, count,orerby,limit 5

customers_main_df.groupBy('cust_lname').count().withColumnRenamed('count','lname_count').orderBy('lname_count',ascending = False).limit(5).show()

+----------+-----------+
|cust_lname|lname_count|
+----------+-----------+
|     Smith|       4626|
|   Johnson|         76|
|  Williams|         69|
|     Jones|         65|
|     Brown|         62|
+----------+-----------+



In [19]:
#  3.3. Check whether there are any customers whose zip codes are not valid (i.e., not equal to 5 digits).

from pyspark.sql.functions import length
customers_main_df.filter(length('cust_zipcode')!=5).count()

0

In [20]:
#  3.4. Check whether there are any customers whose zip codes are valid
customers_main_df.filter(length('cust_zipcode')==5).count()

12435

In [21]:
#  3.5. Find the number of customers from each city in the state of California(CA).
customers_main_df.filter('cust_state = "CA"').groupBy('cust_city').count().orderBy('count',ascending = False).show()

+--------------+-----+
|     cust_city|count|
+--------------+-----+
|   Los Angeles|  224|
|     San Diego|  104|
|      San Jose|   71|
|   Bakersfield|   41|
|     Santa Ana|   36|
|    Long Beach|   36|
|        Fresno|   29|
|     Escondido|   29|
|       Ontario|   29|
| San Francisco|   28|
|     Riverside|   27|
| Mission Viejo|   26|
|     Oceanside|   24|
|    Sacramento|   23|
|       Modesto|   23|
|       Fremont|   22|
|       Hayward|   21|
| Moreno Valley|   21|
|   Simi Valley|   20|
|San Bernardino|   20|
+--------------+-----+
only showing top 20 rows



In [22]:
# spark.stop()